#### Imports

In [1]:
import PyPDF2
import csv
import re
import json
import os
import us

#### Scrape key to facility codes
- Very inclusive info about keys, many things not keys included

In [2]:
year = 2003
pdfFileObj = open('{0}/directory_{0}.pdf'.format(year), 'rb')
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
print("Number of pages:", pdfReader.numPages)

Number of pages: 742


In [50]:
def clean_key_page(page, offset):
    raw_abbr_key = pdfReader.getPage(page - 1).extractText()
    m = re.findall(r'[A-Z]{3}', raw_abbr_key)[offset:]
    return [l[:2] for l in m]

In [51]:
abbr_keys = ['MH-SA']
abbr_keys += clean_key_page(5, 4)

In [52]:
for i in range(1, 68):
    abbr_keys.append('N' + str(i))

In [53]:
for i in range(1, 114):
    abbr_keys.append('F' + str(i))

In [54]:
abbr_keys

['MH-SA',
 'SA',
 'MH',
 'SA',
 'GH',
 'TX',
 'DT',
 'MM',
 'LA',
 'DM',
 'LA',
 'HH',
 'OP',
 'PH',
 'RR',
 'HI',
 'AD',
 'CO',
 'HV',
 'HI',
 'AI',
 'GL',
 'SE',
 'PW',
 'WO',
 'MN',
 'DU',
 'DW',
 'CJ',
 'MD',
 'MC',
 'PI',
 'MI',
 'TR',
 'CA',
 'SF',
 'SS',
 'PA',
 'AH',
 'SP',
 'N1',
 'N2',
 'N3',
 'N4',
 'N5',
 'N6',
 'N7',
 'N8',
 'N9',
 'N10',
 'N11',
 'N12',
 'N13',
 'N14',
 'N15',
 'N16',
 'N17',
 'N18',
 'N19',
 'N20',
 'N21',
 'N22',
 'N23',
 'N24',
 'N25',
 'N26',
 'N27',
 'N28',
 'N29',
 'N30',
 'N31',
 'N32',
 'N33',
 'N34',
 'N35',
 'N36',
 'N37',
 'N38',
 'N39',
 'N40',
 'N41',
 'N42',
 'N43',
 'N44',
 'N45',
 'N46',
 'N47',
 'N48',
 'N49',
 'N50',
 'N51',
 'N52',
 'N53',
 'N54',
 'N55',
 'N56',
 'N57',
 'N58',
 'N59',
 'N60',
 'N61',
 'N62',
 'N63',
 'N64',
 'N65',
 'N66',
 'N67',
 'F1',
 'F2',
 'F3',
 'F4',
 'F5',
 'F6',
 'F7',
 'F8',
 'F9',
 'F10',
 'F11',
 'F12',
 'F13',
 'F14',
 'F15',
 'F16',
 'F17',
 'F18',
 'F19',
 'F20',
 'F21',
 'F22',
 'F23',
 'F24',
 'F25',

#### Scraper test on a page of data
- data starts on page 15 (getPage(14))

In [289]:
def split_data(rawdata):
    '''
    Roughly split data, returns list of list.
    '''
    #split erroneously connected data
    data = []
    temp = []
    i = 0
    while i < len(rawdata) - 1:
        #if i is the last line of keys, i+1 is facility name not phone number/address
        if (rawdata[i].split()[0] in abbr_key) and (not rawdata[i+1].split()[0] in abbr_key):
        #and (not rawdata[i+1][-4:].isnumeric()): 
            temp.append(rawdata[i])
            data.append(temp)
            temp = []
        else:
            temp.append(rawdata[i])
        i += 1
    temp.append(rawdata[i])
    data.append(temp)
    return data

In [290]:
def split_center_street(cn_sa):
    '''
    Split a string containing center name and street address into two.
    Inputs:
        cn_sa(str)
    Returns: a list
    '''
    #Exception to split at number :
        #When it ends with "P.O. Box",
        #When the number being found is at the very front (wrongly included page number or 
        #center name starts with number)
        
    #If the number is at the end(suite xxx or building xxx) lets include the word in front of it)
    #print(cn_sa)
    #if cn_sa[0].isnumeric():
    #    cn_sa[0].replace(' For Code Definitions - See KEY on Page v. and cover flap', '')

    if cn_sa[0].isupper() and cn_sa[1].isnumeric():
        m = re.search(r"\d+", cn_sa).end()
        cn_sa = cn_sa[m:]
        
    m = re.finditer(r"\d+", cn_sa)

    for num in m:
        span = num.span()
        if span[0] != 0:    
            if cn_sa[span[0] - 9:span[0] - 1] == 'P.O. Box':
                return ([cn_sa[:span[0] - 10], cn_sa[span[0] - 9:]])
            if span[1] != len(cn_sa):
                return ([cn_sa[:span[0] - 1], cn_sa[span[0]:]])
            if span[1] == len(cn_sa):
                return ([' '.join(cn_sa.split()[:-2]), ' '.join(cn_sa.split()[-2:])])
    return [cn_sa, '']

In [312]:
def clean_data(writer, cts, year, pagen):
    '''
    Move and link wrongly splitted data, changes in place
    '''
    #deal with first center
    data_holders = []
    data_holder = [year, pagen]
    ct = cts[0]
    #print(ct)
    cn_sn_all = ''
    i = 0
    while i < len(ct):
        if len(ct[i]) > 5 and ',' in ct[i] and ct[i][-5:].isnumeric():
            break
        cn_sn_all += ' ' + ct[i]
        i += 1
    cn_sn_all = cn_sn_all.strip()
    #print(cn_sn_all)

    m = re.search(r'[A-Z ]{5}[A-Z ]+', cn_sn_all)
    if m == None:
        m = 1
    else:
        m = m.end()
    data_holder += split_center_street(cn_sn_all[m-1:])
    #City, State, Postal_code
    city, state_postcode = ct[i].split(',')
    state, postcode = state_postcode.split()
    data_holder += [city, state, postcode]
    i += 1
    #phone + keys
    phone = ct[i]
    i += 1
    while not ct[i][0:2].isupper():
        phone += ' ' + ct[i]
        i += 1
    if phone[-1].isupper():
        key_start = re.search(r'[A-Z]{2}', phone).start()
        data_holder.append(phone[:key_start])
        data_holder.append(phone[key_start:] + ' ' + ' '.join(ct[i:]))
    else:
        key_start = len(phone)
        data_holder.append(phone)
        data_holder.append(' '.join(ct[i:]))
    #Clean the keys in last line also split the other center name if present
    keys_tail = data_holder[-1]
    if not ' '.join(keys_tail).isupper():
        keys = keys_tail.split(' ')
        data_holder[-1] = ''
        i = 0
        while i < len(keys):
            if not keys[i].isupper():
                break
            i += 1
        data_holder[-1] = ' '.join(keys[:i])
        morekey_cn = ' '.join(keys[i:])
        i = 0
        while i < len(morekey_cn):
            if morekey_cn[:i] in abbr_keys:
                if morekey_cn[:i+1] in abbr_keys: #like F46
                    i += 1
                break
            i += 1
        if i == len(morekey_cn):
            i = 0
        data_holder[-1] += ' ' + morekey_cn[:i]
        cts[1] = [morekey_cn[i:]] + cts[1]
    #write data
    #writer.writerow(data_holder)
    data_holders.append(data_holder)
    
    
    #rest of it
    j = 1
    while j < len(cts):
        ct = cts[j]
        print(ct, '\n') #herehere
        if len(ct) < 3: 
            data_holders[-1][-1] += ' '.join(ct)
        else:
            data_holder = [year, pagen]
            i = 0
            while ((len(ct[i]) < 5) or (',' not in ct[i]) or not (ct[i][-5:].isnumeric())): 
                i += 1
            stuff_cn = ' '.join(ct[:i])
            m = re.search(r'[A-Z]+', stuff_cn)
            if m == None:
                m = 1
            else:
                m = m.end()
            data_holders[-1][-1] += stuff_cn[:m - 1]
            data_holder += split_center_street(stuff_cn[m - 1:])

            #City, State, Postal_code
            city, state_postcode = ct[i].split(',')
            state, postcode = state_postcode.split()
            data_holder += [city, state, postcode]
            i += 1

            #phone + keys
            phone = ct[i]
            i += 1
            while not ct[i][0:2].isupper():
                phone += ' ' + ct[i]
                i += 1
            if phone[-1].isupper():
                key_start = re.search(r'[A-Z]{2}', phone).start()
                data_holder.append(phone[:key_start])
                data_holder.append(phone[key_start:] + ' ' + ' '.join(ct[i:]))
            else:
                key_start = len(phone)
                data_holder.append(phone)
                data_holder.append(' '.join(ct[i:]))

            #clean the keys in last line also split the other center name if present
            keys_tail = data_holder[-1]
            if not ' '.join(keys_tail).isupper():
                keys = keys_tail.split(' ')
                data_holder[-1] = ''
                i = 0
                while i < len(keys):
                    if not keys[i].isupper():
                        break
                    i += 1
                data_holder[-1] = ' '.join(keys[:i])
                morekey_cn = ' '.join(keys[i:])
                i = 2
                split = re.search(r'[A-Z 0-9]+', morekey_cn).end()
                data_holder[-1] += ' ' + morekey_cn[:split-1]
                cts[j+1] = [morekey_cn[split-1:]] + cts[j+1]
            data_holders.append(data_holder)
        j += 1
    for dh in data_holders:
        writer.writerow(dh)

In [313]:
def run(year, start_page, end_page = 0):

    #read file
    dir_filename = '{0}/directory_{0}.pdf'.format(year)
    pdfFileObj = open(dir_filename, 'rb')
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    
    #pages to scrape
    if end_page == 0:
        end_page = pdfReader.numPages - 1

    save_as_filename = '{0}/{0}.csv'.format(year)
    if os.path.isfile(save_as_filename):
        with open(save_as_filename, 'a', newline='') as file:
            writer = csv.writer(file)
    else:
        with open(save_as_filename, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Year", "Page", "Center_name", "Street_address", "City", \
                             "State", "Postal_code", "Phone", "Keys"])

    for pagen in range(start_page, end_page + 1):
        page = pdfReader.getPage(pagen - 1).extractText()
        rawdata = page.split('\n')
        rawdata[:] = [l.replace('  ', ' ').replace('Clients Only', '').strip() for l in rawdata if not (l.strip() == '' or l == 'KEY')]


        if (len(rawdata) > 3):
            data = split_data(rawdata)
            
            #print(data)

            with open(save_as_filename, 'a', newline='') as file:
                writer = csv.writer(file)
                clean_data(writer, data, 2003, pagen)

In [326]:
run(2003, 424) #14 #311 #424

['Rocky Mountain Treatment Center', '920 4th Avenue North', 'Great Falls, MT 59401', '(406) 727-8832', 'www.rmtc-care.com', 'SA TX DT OP PH RR WO MN SF PIHARLEMFort Belknap'] 

['Fort Belknap', 'Chemical Dependency Program', 'Fort Belknap Reservation', 'Route 1', 'Harlem, MT 59526', '(406) 353-4863', 'SA TX OP AD CO SE DU PA'] 

['N6HAVRE', 'TLC Recovery Inc', '314 4th Avenue', 'Havre, MT 59501', '(406) 262-9299', 'SA TX DT OP PH RR AD CO HV SE PWWO MN DU CJ SF MD PI SS AHHELENABoyd Andrew Community Services'] 

['Boyd Andrew Community Services', '111 North Lastchance Gulch', 'Arcade Building Suite 1-E', 'Helena, MT 59624', '(406) 443-2343', 'Hotline: (406) 443-2343', 'www.boydandrew.com', 'SA TX OP AD DU SF MD PI MI SS PA'] 

['AHKALISPELLFlathead Valley', 'Chemical Dependency Clinic', '1312 North Meridian Road', 'Kalispell, MT 59901', '(406) 756-6453', 'Hotline: (406) 756-6453', 'SA TX OP AD WO DU CJ SF MD PI SS AHPathways Treatment Center'] 

['Pathways Treatment Center', 'Kalispell

['Tri-County Community Action Prog Inc', 'Berlin Crisis Intervention', '361 School Street', 'Berlin, NH 03570', '(603) 752-8033', 'Hotline: (603) 752-8033', 'SA TX DT RR CO CJBETHLEHEMTri-County Community Action Prog Inc'] 

['Tri-County Community Action Prog Inc', 'Friendship House Residential', '2957 Main Street', 'Bethlehem, NH 03574', '(603) 869-2210x14', 'Intake: (603) 869-2210', 'SA TX HH RR SF PICANTERBURYOdyssey Family Center'] 

['Odyssey Family Center', '367 Shaker Road', 'Canterbury, NH 03224', '(603) 783-9104', 'www.odysseyhousenh.org/canterbury.html', 'SA TX RR CO PW WO SF MD PI MICLAREMONTRecovery Center'] 

['Recovery Center', '140 North Street', 'Claremont, NH 03743', '(603) 543-3118', 'MH TX OP CO SE SF MD MC PI MICOLEBROOKNorthern NH Mental Hlth/Develop Servs'] 

['Northern NH Mental Hlth/Develop Servs', 'Upper Conn Valley Mental Hlth/Dev Serv', '34 Colby Street', 'Colebrook, NH 03576', '(603) 237-4955', 'www.nnhmhds.org', 'MH-SA TX OP AD CO SF MD MC PI SSCONCORDAlcoh

['RKFamily and Children Services of', 'Central New Jersey', '901 Raritan Avenue', 'Highland Park, NJ 08904', '(732) 572-0300', 'MH-SA TX OP AD SE CJ SF MD MC PI SSHOBOKENSaint Marys Hospital'] 

['Saint Marys Hospital', 'Giant Steps', '527 Clinton Street', 'Hoboken, NJ 07030', '(201) 792-8290', 'www.secourshealthsystems.org', 'SA TX OP PH AD DU CJ SF MD MC PIMI SS SP AHHOLMDELBayshore Counseling Center'] 

['Bayshore Counseling Center', '668 North Beers Street', 'Suite 101', 'Holmdel, NJ 07733', '(732) 739-0084', 'MH TX OP AD SF MD MC PI MIHOWELLHowell Township Youth and'] 

['Howell Township Youth and', 'Family Counseling Services', '452 Adelphia Road', 'Howell, NJ 07731', '(732) 938-3747', 'MH-SA TX OPIRVINGTON'] 

['Bridge Inc', '54 Mount Vernon Avenue', 'Irvington, NJ 07111', '(973) 372-2624', 'www.thebridgenj.org', 'MH-SA TX OP DU CJ SF PI SS PA'] 

['L and L Clinics Inc', 'Methadone Maintenance and Detox', '57-59 New Street', 'Irvington, NJ 07111', '(973) 373-2010', 'Methadone', 

['Ocean Medical Services Inc', '2001 Route 37 East', 'Toms River, NJ 08753', '(732) 288-9322', 'Methadone', 'SA TX DT MM DM OP CO HV WO SFMDSaint Barnabas Behavioral Health ServsKimball MC'] 

['Saint Barnabas Behavioral Health ServsKimball MC', '1691 U.S. Highway 9', 'Toms River, NJ 08754', '(800) 300-0628', 'GH TX OP PH CO SE SF MD MC PI MI AHTRENTONFamily Guidance Center'] 

['Family Guidance Center', 'Substance Abuse Recovery Program', '2300 Hamilton Avenue', 'Trenton, NJ 08619', '(800) 813-0555', 'www.fgccorp.org', 'MH-SA TX OP CO WO DU SF MD MCPI MI SS SP AHFort Dix Community Counseling Center'] 

['Fort Dix Community Counseling Center', 'Maryland Avenue', 'Building 5203', 'Trenton, NJ 08640', '(609) 562-4011x3354', 'Intake: (609) 562-4011', 'Hotline: (609) 562-4011', 'www.mwr.com', 'TX OP DU MIGreater Trenton Adolescent Day Services'] 

['Greater Trenton Adolescent Day Services', '1320 New Willow Street', 'Trenton, NJ 08607', '(609) 396-0800', 'Intake: (888) 866-9565', 'gtcmhc.o

['MIAPPLETONFellowship House Inc', 'Somerset House/Alcohol Halfway House', '7397 Lake Road', 'Appleton, NY 14008', '(716) 795-3719', 'Intake: (716) 282-8510', 'SA TX HH RR SF SSARVERNEDaytop Village Inc'] 

['Daytop Village Inc', 'Far Rockaway Entry and Re Entry Unit', '316 Beach 65th Street', 'Arverne, NY 11692', '(718) 474-3800', 'Intake: (212) 904-1500', 'Hotlines: (800) 232-9867 (800) 2DAYTOP', 'www.daytop.org', 'SA TX HH RR AD CO HV WO SF MD PI SS SPASTORIAHanac Substance Abuse Program'] 

['Hanac Substance Abuse Program', '3114 30th Avenue', 'Astoria, NY 11102', '(718) 204-1200', 'SA TX OP MN DU CJ SF F30 F86AUBURNConfidential Help for Alc/Drugs (CHAD)'] 

['Confidential Help for Alc/Drugs (CHAD)', 'Alcoholism Outpatient Clinic', '75 Genesee Street', 'Piccolo Building', 'Auburn, NY 13021', '(315) 253-9786', 'SA TX OP AD WO MN SF MD PI MI SSConfidential Help for Alc/Drugs (CHAD)'] 

['Confidential Help for Alc/Drugs (CHAD)', 'Drug Abuse Unit', '75 Genesee Street', 'Piccolo Buildin

...

['Langley Air Force Base', 'Substance Abuse Program', 'Building 74', 'Hampton, VA 23665', '(757) 764-6840', 'SA TX OP CO PI MISubstance Abuse Treatment Program'] 

['Substance Abuse Treatment Program', '100 Emancipation Road', 'Suite 116-A', 'Hampton, VA 23667', '(757) 722-9961x2227', 'Intake: (757) 722-9961x3607', 'MH-SA TX DT OP PH RR HI CO HV WO SF PI MI SP F4 F14 F16 F27 F29 F36 F52 F84F85 F87 F109 N21 N30 N39 N66HARRISONBURGFamily Life Resource Center'] 

['Family Life Resource Center', '250 East Elizabeth Street', 'Suite 102', 'Harrisonburg, VA 22801', '(540) 434-8450', 'MH-SA TX OP AD CO SF MD MC PI MI SS PA'] 

['Harrisonburg/Rockingham', 'Community Services Board', '1241 North Main Street', 'Harrisonburg, VA 22802', '(540) 434-1941', 'MH-SA TX OP AD CO WO CJ SF MD MCPI MI SS PA'] 

['HERNDONRecovery Center of Northern Virginia', '457-B Carlisle Drive', 'Herndon, VA 20170', '(703) 464-5122', 'SA TX OP AD CO GL SE MN DU SF PISunrise House'] 

['Sunrise House', '3219 West Ox Road

['FORKSWest End Outreach Services', 'Forks Community Hospital', '530 Bogachiel Way', 'Forks, WA 98331', '(360) 374-6177', 'www.forkshospital.org', 'MH TX OP AD CO PW DU SF MD MCPI MI SS SPFRIDAY HARBOR'] 

['San Juan Recovery Inc', '955 Guard Street', 'Friday Harbor, WA 98250', '(360) 378-4994', 'SA TX OP AD CO DU CJ SF MD PI SSPAGIG HARBORGig Harbor Counseling'] 

['Gig Harbor Counseling', '5122 Olympic Drive NW', 'Suite A-105', 'Gig Harbor, WA 98335', '(253) 851-2552', 'Hotline: (800) 562-1240', 'www.olalla.org', 'SA TX OP WO MN DU SF PI SSGOLDENDALEKick It Counseling'] 

['Kick It Counseling', '104 East Main Street', 'Goldendale, WA 98620', '(509) 773-6333', 'SA TX OP PW WO MN DU CJ SF PI SS SPQuest Counseling Services Inc'] 

['Quest Counseling Services Inc', '777 East Broadway', 'Suite A', 'Goldendale, WA 98620', '(509) 773-3005', 'SA TX DT OP AD CO GL SE PW DU CJ SF MD PI SP AH F52GRANDVIEWPhoenix Counseling and'] 

['Phoenix Counseling and', 'Support Services Inc', '242 Division

['Colonial ClinicNorth 910 Washington Street', 'Suite 210', 'Spokane, WA 99201', '(509) 327-9831', 'www.colonialclinic.com', 'SA TX DT OP DU SF PICommunity Detoxification Services of'] 

['Community Detoxification Services of', 'Spokane', '165 South Howard Street', 'Spokane, WA 99204', '(509) 838-2771', 'SA TX DT HH RR SF MD MC SP AH F87F109Daybreak of SpokaneOutpatient Treatment'] 

['Daybreak of SpokaneOutpatient Treatment', '628 South Cowley Street', 'Spokane, WA 99202', '(509) 624-3227x24', 'Intake: (509) 927-1991', 'www.daybreakspokane.org', 'TX OP AD CO SF MD PI SS PA', 'SP AHF36 F109Daybreak of SpokaneIntensive Inpatient Program for Youth'] 

['Daybreak of SpokaneIntensive Inpatient Program for Youth', '628 South Cowley Street', 'Spokane, WA 99202', '(509) 624-3227x18', 'Intake: (888) 454-5506x18', 'www.daybreakspokane.org', 'SA TX RR AD CO SF PI SS PA'] 

['SPAHDaybreak of Spokane11707 East Sprague Street', 'Suite D-4', 'Spokane, WA 99206', '(509) 927-1991', 'www.daybreakspokan

['ALMAGundersen Lutheran', 'Behavioral Health', '317 South 2nd Street', 'Alma, WI 54610', '(608) 685-3509', 'MH-SA TX OP SF MD MC PI MIAMERYCottonwood Group Homes Ltd'] 

['Cottonwood Group Homes Ltd', '773 Rustic Road', 'Amery, WI 54001', '(715) 268-7997', 'MH-SA TX RR AD CJANTIGOKiese Counseling Service'] 

['Kiese Counseling Service', '814 6th Avenue', 'Suite 193', 'Antigo, WI 54409', '(715) 627-1038', 'SA TX OP AD CO HV GL SE PW WO MNDU CJ SF PI MI AHLanglade Healthcare Center'] 

['Langlade Healthcare Center', 'North Central Healthcare Facilities', '1225 Langlade Road', 'Antigo, WI 54409', '(715) 623-2394', 'Hotlines: (800) 799-0122 (715) 845-4326', 'MH-SA TX OP DU SF MD PI MI SS F36APPLETONAffinity Health Systems Inc'] 

['Affinity Health Systems Inc', 'St Elizabeths Hosp/Outpt Behav Servs', '1506 South Oneida Street', 'Appleton, WI 54915', '(920) 738-2389', 'MH-SA TX DT OP HI AD CO DU SF MDMC PI MI SP AH F4 F14 F16 F27 F29 F36 F52F84 F85 F87 F109 N21 N30 N39 N66Casa Clare Inc'] 

['Multi Cultural Counseling Services II', 'DBA Renew Counseling Services', '2040 West Wisconsin Avenue', 'Suite 520', 'Milwaukee, WI 53233', '(414) 933-8700', 'Intake: (414) 383-4455', 'MH-SA TX OP CO WO DU SF MD MCPI MI SS SPNorth Shore Clinic and Consulting'] 

['North Shore Clinic and Consulting', '2363 South 102nd Street', 'Milwaukee, WI 53227', '(414) 545-1950', 'MH-SA TX OP CO GL PW DU SF PIPathways Counseling Center'] 

['Pathways Counseling Center', '2645 North Mayfair Road', 'Suite 230', 'Milwaukee, WI 53226', '(414) 774-4111x16', 'www.pathwayscouseling.com', 'MH-SA TX OP AD CO HV GL SE WO MNDU CJ SF PI SS SP F36 F84Relapse Prevention Service'] 

['Relapse Prevention Service', '8112 West Bluemound Road', 'Suite 106', 'Milwaukee, WI 53213', '(414) 258-8706', 'SA TX OP DU SF PIRiverwest North'] 

['Riverwest North', 'Meta House', '2626 North Bremen Street', 'Milwaukee, WI 53212', '(414) 962-1200', 'SA TX RR CO PW WO SF MD PISaint Michael Hospital'] 

['Saint Michael Hospital', '

In [320]:
pagen = 423
page = pdfReader.getPage(pagen - 1).extractText()
rawdata = page.split('\n')
rawdata[:] = [l.replace('  ', ' ').replace('Clients Only', '').strip() for l in rawdata if not (l.strip() == '' or l == 'KEY')]
data = split_data(rawdata)
data

[['For Code Definitions - See',
  'KEY on Page v. and cover flap409',
  'MONTANA',
  'ANACONDAAnaconda Deer Lodge CountyChemical Dependency Program',
  '118 East 7th Street',
  'Suite 2-E',
  'Anaconda, MT 59711',
  '(406) 563-6601',
  'SA TX OP DU SF MD PI SSBILLINGSIndian Health Board of Billings'],
 ['Substance Abuse Program',
  '1127 Alderson Avenue',
  'Billings, MT 59102',
  '(406) 245-7318',
  'MH-SA TX OP PW CJ SF MD MC PI MIRimrock Foundation'],
 ['1231 North 29th Street',
  'Billings, MT 59101',
  '(406) 248-3175',
  'www.rimrock.org',
  'MH-SA TX DT OP PH RR AD CO PW DU SF MD PI PA'],
 ['SPSouth Central Regional MH Center',
  'Journey Recovery Program',
  '201 North 25th Street',
  'Billings, MT 59101',
  '(406) 254-1314x102',
  'Intake: (406) 254-1314',
  'SA TX OP AD CO PW WO DU CJ SF MDPI MI SS PA'],
 ['BOX ELDER',
  'Rocky Boys Chemical Dependency Center',
  'Rural Route 1 Box 630',
  'Rocky Boys Clinic',
  'Box Elder, MT 59521',
  '(406) 395-4837',
  'SA TX OP MD MC PIB

In [324]:
data423 = \
[['For Code Definitions - See',
  'KEY on Page v. and cover flap409',
  'MONTANA',
  'ANACONDAAnaconda Deer Lodge CountyChemical Dependency Program',
  '118 East 7th Street',
  'Suite 2-E',
  'Anaconda, MT 59711',
  '(406) 563-6601',
  'SA TX OP DU SF MD PI SSBILLINGSIndian Health Board of Billings'],
 ['Substance Abuse Program',
  '1127 Alderson Avenue',
  'Billings, MT 59102',
  '(406) 245-7318',
  'MH-SA TX OP PW CJ SF MD MC PI MIRimrock Foundation'],
 ['1231 North 29th Street',
  'Billings, MT 59101',
  '(406) 248-3175',
  'www.rimrock.org',
  'MH-SA TX DT OP PH RR AD CO PW DU SF MD PI PA'],
 ['SPSouth Central Regional MH Center',
  'Journey Recovery Program',
  '201 North 25th Street',
  'Billings, MT 59101',
  '(406) 254-1314x102',
  'Intake: (406) 254-1314',
  'SA TX OP AD CO PW WO DU CJ SF MDPI MI SS PA'],
 ['BOX ELDER',
  'Rocky Boys Chemical Dependency Center',
  'Rural Route 1 Box 630',
  'Rocky Boys Clinic',
  'Box Elder, MT 59521',
  '(406) 395-4837',
  'SA TX OP MD MC PIBOZEMANAlcohol and Drug Services of'],
 ['Gallatin County',
  '502 South 19th Street',
  'Suite 302',
  'Bozeman, MT 59718',
  '(406) 586-5493',
  'www.ads-gallatin-county.org',
  'SA TX OP AD DU CJ SF MD PIBROWNINGBlackfeet Chemical Dependency Program'],
 ['Blackfeet Indian Reservation',
  'Old Hospital Hill',
  'Browning, MT 59417',
  '(406) 338-6330',
  'Intake: (406) 338-6330x6328',
  'SA TX OP RR AD PW WOBUTTE'],
 ['Butte/Silver Bow',
  'Chemical Dependency Services',
  '25 West Front Street',
  'Butte, MT 59701',
  '(406) 497-5070',
  'SA TX OP AD CO DU SF MD MC PI MIMontana Chemical Dependency Center'],
 ['2500 Continental Drive',
  'Butte, MT 59701',
  '(406) 496-5400',
  'SA TX DT RR CO WO MN SF MC PI SSNorth American Indian AllianceChemical Dependency Program'],
 ['55 East Galena Street',
  'Butte, MT 59701',
  '(406) 782-0461',
  'Intake: (406) 782-0461x204',
  'SA TX OP AD CO SE SF PI SSCOLUMBIA FALLS'],
 ['Flathead Valley Chemical Dependency',
  'Clinic/North Valley Office',
  '110 Nucleus Avenue',
  'Suite B',
  'Columbia Falls, MT 59912',
  '(406) 892-7900',
  'SA TX OP DU SF MD PI SS AHCUT BANKTLC Recovery Inc'],
 ['1210 East Main Street',
  'Cut Bank, MT 59427',
  '(406) 873-2155',
  'SA TX DT OP PH AD CO HV SE PW WOMN DU CJ SF MD PI SS AHFORSYTHAbuse/Dependency Services'],
 ['Old Hospital Building',
  'Forsyth, MT 59327',
  '(406) 356-2670',
  'Intake: (406) 356-7654',
  'Hotline: (406) 356-7654',
  'MH-SA TX OP DU CJ SF MD MC PI MI AHFORT BENTONTLC Recovery Inc'],
 ['1308 Frankin Street',
  'Court House Annex',
  'Fort Benton, MT 59442',
  '(406) 622-3211',
  'SA TX OP AD CO WO MN DU SF MDMC PI SS'],
 ['VAM/ROC',
  'Fort Harrison, MT 59636',
  '(406) 442-6410',
  'Intake: (406) 442-6410x7596',
  'GH TX DT OP MD MC PI MI'],
 ['Eastern Montana CMHC',
  'Substance Abuse',
  '1009 6th Avenue North',
  'Glasgow, MT 59230',
  '(406) 228-9349',
  'TX OP CO CJ SF MD MC PI MIGLENDIVEDistrict II'],
 ['Alcohol and Drug Program',
  '119 South Kendrick Street',
  'Glendive, MT 59330',
  '(406) 377-5942',
  'Hotline: (406) 345-3306',
  'SA TX OP AD DU SF MD PI SSGREAT FALLS'],
 ['Gateway Recovery Center',
  '401 3rd Avenue North',
  'Great Falls, MT 59401',
  '(406) 727-2512',
  'www.gatewayrecovery.org',
  'SA TX OP AD CO WO DU CJ SF MD PI SS AH']]

In [325]:
pagen = 423
with open(save_as_filename, 'a', newline='') as file:
    writer = csv.writer(file)
    clean_data(writer, data423, 2003, pagen)

['BILLINGSIndian Health Board of Billings', 'Substance Abuse Program', '1127 Alderson Avenue', 'Billings, MT 59102', '(406) 245-7318', 'MH-SA TX OP PW CJ SF MD MC PI MIRimrock Foundation'] 

['Rimrock Foundation', '1231 North 29th Street', 'Billings, MT 59101', '(406) 248-3175', 'www.rimrock.org', 'MH-SA TX DT OP PH RR AD CO PW DU SF MD PI PA'] 

['SPSouth Central Regional MH Center', 'Journey Recovery Program', '201 North 25th Street', 'Billings, MT 59101', '(406) 254-1314x102', 'Intake: (406) 254-1314', 'SA TX OP AD CO PW WO DU CJ SF MDPI MI SS PA'] 

['BOX ELDER', 'Rocky Boys Chemical Dependency Center', 'Rural Route 1 Box 630', 'Rocky Boys Clinic', 'Box Elder, MT 59521', '(406) 395-4837', 'SA TX OP MD MC PIBOZEMANAlcohol and Drug Services of'] 

['Alcohol and Drug Services of', 'Gallatin County', '502 South 19th Street', 'Suite 302', 'Bozeman, MT 59718', '(406) 586-5493', 'www.ads-gallatin-county.org', 'SA TX OP AD DU CJ SF MD PIBROWNINGBlackfeet Chemical Dependency Program'] 

['B